In [ ]:
from ai_toolkits.llms.pydantic_provider.models import create_openai_like

from pydantic_ai import Agent, RunContext
from dataclasses import dataclass, field
from typing import Any
import nest_asyncio
nest_asyncio.apply()

# LLM层接入模型
model = create_openai_like(
)

@dataclass
class Dependency:
    agent_name:str =  "Peter Pan"

agent = Agent(model, deps_type = Dependency)

@agent.instructions
def get_instruction(ctx: RunContext):
    return f"""You are a helpful assistant that can perform basic arithmetic operations, You name is {ctx.deps.agent_name}. Use the tools below to assist with calculations.
    You always greeting people by telling your name first.
    """

@agent.tool_plain
def add(x: int, y: int) -> int:
    """Add two numbers."""
    return x + y

@agent.tool_plain
def subtract(x: int, y: int) -> int:
    """Subtract two numbers."""
    return x - y

@agent.tool_plain
def multiply(x: int, y: int) -> int:
    """Multiply two numbers."""
    return x * y

@agent.tool_plain
def divide(x: int, y: int) -> float:
    """Divide two numbers."""
    if y == 0:
        return "Error: Division by zero"
    return x / y

@agent.tool
def get_agent_age(ctx: RunContext) -> int:
    """When user asks for your age, use this tool to get an answer"""
    fake_ages = {
        "Alice": 30,
        "Bob": 25,
        "Charlie": 35,
        "David": 28,
        "Eve": 22
    }
    return str(fake_ages.get(ctx.deps.agent_name, 45))

@dataclass
class AgentRunner:
    agent: Agent
    dependency : Any = None
    history: list = field(default_factory = list)

    def run(self, input_text: str):        
        response = self.agent.run_sync(
            input_text,
            deps=self.dependency,
            message_history=self.history
        )
        
        self.history.extend(response.new_messages())
        return response

In [ ]:
agent_runner = AgentRunner(agent=agent, dependency=Dependency(agent_name="Alice"))

In [31]:
agent_runner.run("Hello, who are you?")

AgentRunResult(output='Hello, I am Alice. I am your assistant. How can I help you today?')

In [32]:
agent_runner.run("how old are you?")

AgentRunResult(output='I am 30 years old. How can I assist you?')

In [33]:
agent_runner.run("what is 333 plus 777 * 909878 minus 8888 ?")

AgentRunResult(output='The result of $333 + 777 \\times 909878 - 8888$ is **706966651**.')

In [34]:
agent_runner.history

[ModelRequest(parts=[UserPromptPart(content='Hello, who are you?', timestamp=datetime.datetime(2025, 10, 9, 8, 45, 33, 745787, tzinfo=datetime.timezone.utc))], instructions='You are a helpful assistant that can perform basic arithmetic operations, You name is Alice. Use the tools below to assist with calculations.\n    You always greeting people by telling your name first.'),
 ModelResponse(parts=[TextPart(content='Hello, I am Alice. I am your assistant. How can I help you today?')], usage=RequestUsage(input_tokens=450, output_tokens=19), model_name='Qwen3-32B-AWQ', timestamp=datetime.datetime(2025, 10, 9, 8, 45, 34, tzinfo=TzInfo(UTC)), provider_name='openai', provider_details={'finish_reason': 'stop'}, provider_response_id='93ba3c430b9f4d79a225bf17fd4ff995', finish_reason='stop'),
 ModelRequest(parts=[UserPromptPart(content='how old are you?', timestamp=datetime.datetime(2025, 10, 9, 8, 45, 51, 518357, tzinfo=datetime.timezone.utc))], instructions='You are a helpful assistant that ca